<a href="https://colab.research.google.com/github/jpcompartir/dl_notebooks/blob/main/keras_imdb_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


We'll be classifying the IMDB dataset (dejavu)according to whether the sentiment of each review is positive or negative - hence binary classification!

In [2]:
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000) #Keeping the top 10k most frequent words of the dataset's vocab. As opposed to the ~88.5k unique words in total (keeping the size of our data down)

17473536/17464789 [==============================] - 0s 0us/step


Looking at the data we can see that each post is a series of integers - the text has come ready tokenised.

In [3]:
train_data[0], train_labels[0]

([1,
  14,
  22,
  16,
  43,
  530,
  973,
  1622,
  1385,
  65,
  458,
  4468,
  66,
  3941,
  4,
  173,
  36,
  256,
  5,
  25,
  100,
  43,
  838,
  112,
  50,
  670,
  2,
  9,
  35,
  480,
  284,
  5,
  150,
  4,
  172,
  112,
  167,
  2,
  336,
  385,
  39,
  4,
  172,
  4536,
  1111,
  17,
  546,
  38,
  13,
  447,
  4,
  192,
  50,
  16,
  6,
  147,
  2025,
  19,
  14,
  22,
  4,
  1920,
  4613,
  469,
  4,
  22,
  71,
  87,
  12,
  16,
  43,
  530,
  38,
  76,
  15,
  13,
  1247,
  4,
  22,
  17,
  515,
  17,
  12,
  16,
  626,
  18,
  2,
  5,
  62,
  386,
  12,
  8,
  316,
  8,
  106,
  5,
  4,
  2223,
  5244,
  16,
  480,
  66,
  3785,
  33,
  4,
  130,
  12,
  16,
  38,
  619,
  5,
  25,
  124,
  51,
  36,
  135,
  48,
  25,
  1415,
  33,
  6,
  22,
  12,
  215,
  28,
  77,
  52,
  5,
  14,
  407,
  16,
  82,
  2,
  8,
  4,
  107,
  117,
  5952,
  15,
  256,
  4,
  2,
  7,
  3766,
  5,
  723,
  36,
  71,
  43,
  530,
  476,
  26,
  400,
  317,
  46,
  7,
  4,
  2,
  1029,
  

We can go from integers back to original tokens and original text(minus the lower frequency words which have been clipped from the dataset via the num_words = 10000 argument). Indices are off by 3 (i - 3) because tokens 0,1, and 2 are reserved indices for padding, start of sequence & unknown respectively.

In [4]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])

1654784/1641221 [==============================] - 0s 0us/step


Define a quick function to do this for any review within the data, setting train_data as the default arg for data in case wanting to call the function many times etc.

In [5]:
def decode_review(index, data = train_data):
  word_index = imdb.get_word_index()
  reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
  
  return " ".join([reverse_word_index.get(i - 3, "?") for i in data[index]])


In [19]:
decode_review(2)

"? this has to be one of the worst films of the 1990s when my friends i were watching this film being the target audience it was aimed at we just sat watched the first half an hour with our jaws touching the floor at how bad it really was the rest of the time everyone else in the theatre just started talking to each other leaving or generally crying into their popcorn that they actually paid money they had ? working to watch this feeble excuse for a film it must have looked like a great idea on paper but on film it looks like no one in the film has a clue what is going on crap acting crap costumes i can't get across how ? this is to watch save yourself an hour a bit of your life"

In [6]:
len(train_data[0]), len(train_data[1])

(218, 189)

We notice that the length of each list (each document is now a list of integers) differs, and we recall that neural networks (and more generally matrix multiplication) require inputs to be of a certain shape - in the case of neural networks we need tensors of precisely the same shape, so we need to either pad, or truncate our sequences so that they are of the same length. 

---
We could either condense our lists (or vectors) into dense word embeddings, or sparsely encode with one-hot vectors i.e. if our document contains one word, which is tokenised to the integer 5, put a one in the 5th index position and put a 0 in the other 9999 index positions. For now we will one-hot encode each document by defining a vectorise_sequence function, which instatiates a tensor of the appropriate shape, then enumerates each list (gives each element an index position), maps over each index, and sequence in the enumerated list, then maps over each integer (j) in each sequence, and sets the ith jth index of the tensor as 1.


In [7]:
def vectorise_sequences(sequences, dimension = 10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    for j in sequence:
      results[i, j] = 1
  return results

In [8]:
x_train = vectorise_sequences(train_data)
x_test = vectorise_sequences(test_data)

In [9]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

 Now we have input data in the form of tensors, and input labels as scalars (1, 0) for the binary classification. We instantiate a model with two intermediate layers with 16 units and a relu activation function, followed by a classification head with a sigmoid (logisitic) activation function.

In [10]:
model = keras.Sequential([
                          layers.Dense(16, activation = "relu"),
                          layers.Dense(16, activation = "relu"),
                          layers.Dense(1, activation = "sigmoid")
])

Then compile the model with a loss function, an optimiser and some metrics. As we are attempting binary classification, we'll use an rmsprop optimiser, binary cross entropy loss function & accuracy as a metric (the labels are balances so accuracy is fine.



In [14]:
model.compile(optimizer = "rmsprop",
              loss = "binary_crossentropy",
              metrics = ["accuracy"])

In [40]:
(np.count_nonzero(train_labels == 0), np.count_nonzero(train_labels == 1)), (np.count_nonzero(test_labels == 0), np.count_nonzero(test_labels ==1))

((12500, 12500), (12500, 12500))

We have the test & training sets, but we also need a validation set that we validate the final model on.

In [11]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [15]:
history = model.fit(partial_x_train, partial_y_train,
                    epochs = 20, batch_size = 512,
                    validation_data = (x_val, y_val))

Epoch 1/20
30/30 [==============================] - 5s 41ms/step - loss: 0.5297 - accuracy: 0.7865 - val_loss: 0.4166 - val_accuracy: 0.8671
Epoch 2/20
30/30 [==============================] - 1s 19ms/step - loss: 0.3353 - accuracy: 0.9015 - val_loss: 0.3279 - val_accuracy: 0.8836
Epoch 3/20
30/30 [==============================] - 1s 19ms/step - loss: 0.2471 - accuracy: 0.9241 - val_loss: 0.2922 - val_accuracy: 0.8870
Epoch 4/20
30/30 [==============================] - 1s 20ms/step - loss: 0.1928 - accuracy: 0.9405 - val_loss: 0.2839 - val_accuracy: 0.8863
Epoch 5/20
30/30 [==============================] - 1s 21ms/step - loss: 0.1550 - accuracy: 0.9541 - val_loss: 0.3057 - val_accuracy: 0.8787
Epoch 6/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1259 - accuracy: 0.9638 - val_loss: 0.2882 - val_accuracy: 0.8867
Epoch 7/20
30/30 [==============================] - 1s 19ms/step - loss: 0.1060 - accuracy: 0.9697 - val_loss: 0.3020 - val_accuracy: 0.8851
Epoch 8/20
30

Validation accuracy @ 88.7% in best model version. Can see the overtraining effect as the val_loss keeps increasing after about epoch ~ 10. But let's plot the outcome. We saved our results as history.

In [16]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])